### Search Engine with tools and agents

In [1]:
#Arxiv
##tools creation


from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper



import streamlit

In [2]:
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)#tool 
wiki

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/aryamantiwari17/GenAI/Langchain/Basic+advance RAG/myenv1/lib/python3.12/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250))

In [3]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arx=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)#tool 
arx

ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250))

In [4]:
tools=[wiki,arx]

In [5]:
import os
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

In [6]:
##custom tool[RAG tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings


USER_AGENT environment variable not set, consider setting it to identify your requests.
/home/aryamantiwari17/GenAI/Langchain/Basic+advance RAG/myenv1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [7]:
loader=WebBaseLoader("https://huggingface.co/Qwen/Qwen2.5-Omni-7B")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,embedding=embeddings)
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x70efa7f997c0>, search_kwargs={})

In [8]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,'hF-tool',"Search about any thing whuch has qwen")
retriever_tool.name

'hF-tool'

In [9]:
tools=[wiki,arx,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/aryamantiwari17/GenAI/Langchain/Basic+advance RAG/myenv1/lib/python3.12/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='hF-tool', description='Search about any thing whuch has qwen', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x70f0b9d81f80>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object a

In [10]:
#Run all this tools with agents and LLM models
## tools -->agent excutor
from dotenv import load_dotenv
import os
from langchain_groq import ChatGroq
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
llm = ChatGroq(model_name="mistral-saba-24b", temperature=0.7, max_tokens=4096)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x70efa7181970>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x70efa7182a80>, model_name='mistral-saba-24b', model_kwargs={}, groq_api_key=SecretStr('**********'), max_tokens=4096)

In [11]:
#promt template
from langchain import hub
prompt = hub.pull("hwchase17/openai-tools-agent")

prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [12]:
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [13]:
##agent Executor
from langchain.agents import AgentExecutor
agent_exector=AgentExecutor(agent=agent,tools=tools,verbose='True')
agent_exector                        

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [14]:
r=agent_exector.invoke({"input":"tell me about qwen?"})
r




> Entering new AgentExecutor chain...

Invoking: `hF-tool` with `{'query': 'about qwen'}`


Model tree for Qwen/Qwen2.5-Omni-7B




Adapters

2 models


Finetunes

1 model


Quantizations

1 model



						Spaces using
						Qwen/Qwen2.5-Omni-7B
5
🏆
Gyaneshere/Qwen2.5-Omni-7B-MultimodalInput-to-Speech🏆
wambugu71/Qwen2.5-Omni-7B-Demo🏆
aimeri/Qwen2.5-Omni-7B-Demo📊
srinuksv/qwen2.5-omini🐠
srinuksv/f



						Collection including
						Qwen/Qwen2.5-Omni-7B

Qwen2.5-Omni

					Collection
				
End-to-End Omni (text, audio, image, video, and natural speech interaction) model  based Qwen2.5
• 
3 items
• 
Updated
					9 days ago
•


					77







			System theme
		

Company
TOS
Privacy
About
Jobs

Website
Models
Datasets
Spaces
Pricing
Docs

HumanEval
78.7
84.8
79.9
72.6
68.9


MBPP
73.2
79.2
67.2
69.6
74.9


MultiPL-E
65.8
70.4
59.1
50.7
53.4


LiveCodeBench2305-2409
24.6
28.7
23.9
8.3
18.9










		Quickstart
	

Below, we provide simple examples to show how to use Qwen2.5-Omni with 🤗 Tr

{'input': 'tell me about qwen?',
 'output': 'Qwen2.5-Omni-7B is an end-to-end multimodal model designed to perceive diverse modalities, including text, images, audio, and video, while simultaneously generating text and natural speech responses in a streaming manner. It supports both text and audio outputs, and users can set `enable_audio_output=False` in the `from_pretrained` function to save GPU memory.'}

In [15]:
r=agent_exector.invoke({"input":"tell me about LLMs?"})
r




> Entering new AgentExecutor chain...
LLMs, or Large Language Models, are a type of artificial intelligence model designed to understand and generate human language. These models are trained on vast amounts of text data from the internet, up until a certain point in time. They can generate text, translate languages, write creative content, and even answer questions based on the information they have been trained on. However, they do not have real-time web browsing capabilities, personal experiences, or feelings. They operate based on patterns and information present in their training data.

> Finished chain.


{'input': 'tell me about LLMs?',
 'output': 'LLMs, or Large Language Models, are a type of artificial intelligence model designed to understand and generate human language. These models are trained on vast amounts of text data from the internet, up until a certain point in time. They can generate text, translate languages, write creative content, and even answer questions based on the information they have been trained on. However, they do not have real-time web browsing capabilities, personal experiences, or feelings. They operate based on patterns and information present in their training data.'}

In [16]:
r=agent_exector.invoke({"input":"tell me about machine learning?"})
r




> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks witMachine learning is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.

> Finished chain.


{'input': 'tell me about machine learning?',
 'output': 'Machine learning is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.'}

In [17]:
r=agent_exector.invoke({"input":"tell me about some research apapera bout transformer?"})
r




> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'transformer'}`


Published: 2006-12-01
Title: The Xi-transform for conformally flat space-time
Authors: George Sparling
Summary: The Xi-transform is a new spinor transform arising naturally in Einstein's
general relativity. Here the example of conformally flat space-The Xi-transform, introduced by George Sparling, is a novel spinor transform that arises naturally within the framework of Einstein's general relativity. This transform is particularly relevant to conformally flat space-time, which is a special case where the Weyl curvature tensor vanishes. The work published in 2006 provides a detailed exploration of this transform and its implications in the context of general relativity.

> Finished chain.


{'input': 'tell me about some research apapera bout transformer?',
 'output': "The Xi-transform, introduced by George Sparling, is a novel spinor transform that arises naturally within the framework of Einstein's general relativity. This transform is particularly relevant to conformally flat space-time, which is a special case where the Weyl curvature tensor vanishes. The work published in 2006 provides a detailed exploration of this transform and its implications in the context of general relativity."}